kernel: https://www.kaggle.com/the1owl/the-game-of-life

In [22]:
from skimage.morphology import closing, disk, label
from sklearn.preprocessing import MinMaxScaler
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import glob, cv2, math

train = pd.DataFrame(glob.glob('data/stage1_train/**/**/**'), columns=['Path'])
train['ImageId'] = train['Path'].map(lambda x: x.split('/')[4])
train['mask'] = train['Path'].map(lambda x: 1 if 'masks' in x.split('/')[4].lower() else 0)
train_masks = train[train['mask']==1][['Path','ImageId']]
train = train[train['mask']==0][['Path','ImageId']]
train_labels = pd.read_csv('data/stage1_train_labels.csv/stage1_train_labels.csv')
test = pd.DataFrame(glob.glob('data/stage1_test/**/**/**'), columns=['Path'])
test['ImageId'] = test['Path'].map(lambda x: x.split('/')[3])
print(len(train), len(train_masks), len(train_labels), len(test))

30131 0 29461 66


In [23]:
train_masks

,Path,ImageId


In [26]:
train

,Path,ImageId
0,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,6b0ac2ab04c09dced54058ec504a4947f8ecd5727dfca7...
1,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,9484ab39577a3c59658143f8a6d06a19ad7e023ea6213a...
2,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,ae395d4867678a71cb829425265c9bb80094c924c6bccb...
3,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,b5cf84b29c9d2758fbc1b41da1be2cc2650aa36252eb41...
4,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,c1eee5435f6fb9fa62ace9f9fb762dac8fc6b5c26efb20...
5,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,d62987543518e7763e51db6b9f15cc0d1dce95d112d25b...
6,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,f9ffe0d84b20d5f3fa54933fa9cdc188a64b4760e82d23...
7,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,90f94edc66f6dea35c86d82ff4f677b12fbd38ac09b845...
8,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,6604699ac62efaa5527e0a1f16f8870a2f832c5b96da54...
9,data/stage1_train/6b0ac2ab04c09dced54058ec504a...,935e430095e5a0fe1066488c5fc45a9ab785c944635152...
